In [1]:
!rm -rf /Users/ryan/prosodic_data/data/english_wordforms.sqlitedict
import sys; sys.path.insert(0,'..')
from prosodic.imports import *

In [2]:
x="""
Those hours, that with gentle work did frame
The lovely gaze where every eye doth dwell,
Will play the tyrants to the very same
And that unfair which fairly doth excel;
For never-resting time leads summer on
To hideous winter, and confounds him there;
Sap checked with frost, and lusty leaves quite gone,
Beauty o’er-snowed and bareness every where:
Then were not summer’s distillation left,
A liquid prisoner pent in walls of glass,
Beauty’s effect with beauty were bereft,
Nor it, nor no remembrance what it was:
But flowers distill’d, though they with winter meet,
Leese but their show; their substance still lives sweet.
"""
t=Text(x)

In [3]:
def getlenparse(l): return sum(len(x) for x in l)

def iter_mpos(nsyll, starter=[], pos_types=None, maxS=METER_MAX_S, maxW=METER_MAX_W):
    if pos_types is None:
        wtypes = ['w'*n for n in range(1,maxW+1)]
        stypes = ['s'*n for n in range(1,maxS+1)]
        pos_types=[[x] for x in wtypes + stypes]
        
    news=[]
    for pos_type in pos_types:
        if starter and starter[-1][-1]==pos_type[0][0]: continue
        new = starter + pos_type
        # if starter: print(starter[-1][-1], pos_type[0][0], new)
        #if not is_ok_parse(new): continue
        if getlenparse(new)<=nsyll:
            news.append(new)
    
    # news = battle_parses(news)
    if news: yield from news
    # print('\n'.join('|'.join(x) for x in news))
    for new in news: yield from iter_mpos(nsyll, starter=new, pos_types=pos_types)

def get_possible_parses(nsyll):
    return [l for l in iter_mpos(nsyll) if getlenparse(l)==nsyll]

In [4]:
get_possible_parses(2)

[['ww'], ['ss'], ['w', 's'], ['s', 'w']]

In [5]:
# t = Text('hello world')
l = t.lines[0]
slots = l.slot_matrix[0]
# slots

In [6]:
def get_possible_parse_objs(slots, constraints=[]):
    parse_objs = []
    for parse_poss in get_possible_parses(len(slots)):
        slot_i=0
        parse_obj = Parse(len(slots), constraints=constraints)
        mpos_objs=[]
        for mpos in parse_poss:
            mpos_slots=[]
            for slot_mval in mpos:
                mpos_slots.append(slots[slot_i])
                slot_i+=1
            mpos_obj = ParsePosition(mpos[0], parse=parse_obj, slots=mpos_slots)
            mpos_objs.append(mpos_obj)
        # for i in range(1, len(mpos_objs)):
            # mpos_objs[i].prev=mpos_objs[i-1]
            # mpos_objs[i-1].next=mpos_objs[i]
        parse_obj.children = parse_obj.positions = mpos_objs
        parse_objs.append(parse_obj)
    return parse_objs

In [7]:
parses = get_possible_parse_objs(slots)

In [8]:
syll=t.syllables[0]

In [9]:
syll, syll.next, syll.prev

((Syllable: Those) [syll_ipa='ðoʊz syll_stress=P syll_weight=H is_stressed=True is_heavy=True is_strong=None is_weak=None],
 None,
 None)

In [10]:
syll.parent.children.index(syll)

0

In [11]:
syll.children[-1].phon_cons

1

In [12]:
def parse_slots_slow(self, slots, constraints=[]):
    parses = get_possible_parse_objs(slots, constraints=constraints)
    parses, bounded_parses = self.bound_parses(parses)
    parses.sort()
    return parses, bounded_parses

In [13]:
def w_stress(mpos, score=1):
    if not mpos.is_prom and any(slot.is_stressed for slot in mpos.slots): 
        return score

def s_unstress(mpos, score=1):
    if mpos.is_prom and any(not slot.is_stressed for slot in mpos.slots):
        return score
    


def unres_within(mpos, score=1):
    slots = mpos.slots
    if len(slots)!=2: return np.nan
    slot1,slot2=slots
    unit1,unit2=slot1.unit,slot2.unit
    wf1,wf2=unit1.parent,unit2.parent
    if wf1 is not wf2: return np.nan
    # disyllabic position within word
    # first position mist be light and stressed
    if unit1.weight_is_prom or not unit1.stress_is_prom: return score
    return 0
    

def unres_across(mpos, score=1):
    slots = mpos.slots
    if len(slots)!=2: return np.nan
    slot1,slot2=slots
    unit1,unit2=slot1.unit,slot2.unit
    wf1,wf2=unit1.parent,unit2.parent
    if wf1 is wf2: return np.nan
    # disyllabic strong position immediately violates
    if mpos.is_prom: return score
    if not wf1.is_functionword or not wf2.is_functionword: return score
    return 0
    

def w_peak(mpos, score=1):
    if mpos.is_prom: return np.nan
    if any(slot.is_strong for slot in mpos.slots): return score
    return 0

def s_trough(mpos, score=1):
    if not mpos.is_prom: return np.nan
    if any(slot.is_weak for slot in mpos.slots): return score
    return 0

In [14]:
syll = English().get('hours')[0].children[0]
syll

(Syllable: ho) [syll_ipa='aʊ syll_stress=P syll_weight=H is_stressed=True is_heavy=True is_strong=None is_weak=False]

In [15]:
def is_strong(self):
    if not len(self.parent.children)>1: return None
    if not self.is_stressed: return False
    if self.prev and not self.prev.is_stressed: return True
    if self.next and not self.next.is_stressed: return True

In [27]:
syll, syll.next

((Syllable: ho) [syll_ipa='aʊ syll_stress=P syll_weight=H is_stressed=True is_heavy=True is_strong=None is_weak=False],
 (Syllable: urs) [syll_ipa=ɛːz syll_stress=U syll_weight=H is_stressed=True is_heavy=True is_strong=None is_weak=False])

In [17]:
syll.is_stressed

True

In [18]:
syll.is_strong, syll.is_weak

(None, False)

In [19]:
constraints = [w_stress, s_unstress, unres_across, unres_within, w_peak, s_trough]

In [20]:
parses,bparses = parse_slots_slow(l, slots, constraints = constraints)

In [21]:
parses

[(Parse: those|HO|urs|THAT|with|GEN|tle|WORK|did|FRAME) [meter=wswswswsws score=5 w_stress=5],
 (Parse: THOSE|ho|URS|that|WITH|gen|TLE|work|DID.FRAME) [meter=swswswswss score=5 w_stress=4 unres_across=1],
 (Parse: THOSE|ho|URS|that|WITH|gen|TLE|work|DID|frame) [meter=swswswswsw score=5 w_stress=5],
 (Parse: THOSE|ho|URS|that|WITH|gen|TLE|work.did|FRAME) [meter=swswswswws score=5 w_stress=4 unres_across=1],
 (Parse: THOSE|ho|URS|that|WITH|gen|TLE.WORK|did|FRAME) [meter=swswswssws score=5 w_stress=4 unres_across=1],
 (Parse: THOSE|ho|URS|that|WITH|gen.tle|WORK|did|FRAME) [meter=swswswwsws score=5 w_stress=4 unres_within=1],
 (Parse: THOSE|ho|URS|that|WITH.GEN|tle|WORK|did|FRAME) [meter=swswsswsws score=5 w_stress=4 unres_across=1],
 (Parse: THOSE|ho|URS|that.with|GEN|tle|WORK|did|FRAME) [meter=swswwswsws score=5 w_stress=4 unres_across=1],
 (Parse: THOSE|ho|URS.THAT|with|GEN|tle|WORK|did|FRAME) [meter=swsswswsws score=5 w_stress=4 unres_across=1],
 (Parse: THOSE|ho.urs|THAT|with|GEN|tle|

In [22]:
bparses

[(Parse: those|HO|urs|THAT|with|GEN|tle|WORK|did.frame) [meter=wswswswsww score=6 w_stress=5 unres_across=1],
 (Parse: those|HO|urs|THAT|with|GEN|tle|WORK.DID|frame) [meter=wswswswssw score=6 w_stress=5 unres_across=1],
 (Parse: those|HO|urs|THAT|with|GEN|tle.work|DID.FRAME) [meter=wswswswwss score=6 w_stress=4 unres_across=2],
 (Parse: those|HO|urs|THAT|with|GEN|tle.work|DID|frame) [meter=wswswswwsw score=6 w_stress=5 unres_across=1],
 (Parse: those|HO|urs|THAT|with|GEN.TLE|work|DID.FRAME) [meter=wswswsswss score=6 w_stress=4 unres_across=1 unres_within=1],
 (Parse: those|HO|urs|THAT|with|GEN.TLE|work|DID|frame) [meter=wswswsswsw score=6 w_stress=5 unres_within=1],
 (Parse: those|HO|urs|THAT|with|GEN.TLE|work.did|FRAME) [meter=wswswsswws score=6 w_stress=4 unres_across=1 unres_within=1],
 (Parse: those|HO|urs|THAT|with.gen|TLE|work|DID.FRAME) [meter=wswswwswss score=6 w_stress=4 unres_across=2],
 (Parse: those|HO|urs|THAT|with.gen|TLE|work|DID|frame) [meter=wswswwswsw score=6 w_stress

In [23]:
bparses

[(Parse: those|HO|urs|THAT|with|GEN|tle|WORK|did.frame) [meter=wswswswsww score=6 w_stress=5 unres_across=1],
 (Parse: those|HO|urs|THAT|with|GEN|tle|WORK.DID|frame) [meter=wswswswssw score=6 w_stress=5 unres_across=1],
 (Parse: those|HO|urs|THAT|with|GEN|tle.work|DID.FRAME) [meter=wswswswwss score=6 w_stress=4 unres_across=2],
 (Parse: those|HO|urs|THAT|with|GEN|tle.work|DID|frame) [meter=wswswswwsw score=6 w_stress=5 unres_across=1],
 (Parse: those|HO|urs|THAT|with|GEN.TLE|work|DID.FRAME) [meter=wswswsswss score=6 w_stress=4 unres_across=1 unres_within=1],
 (Parse: those|HO|urs|THAT|with|GEN.TLE|work|DID|frame) [meter=wswswsswsw score=6 w_stress=5 unres_within=1],
 (Parse: those|HO|urs|THAT|with|GEN.TLE|work.did|FRAME) [meter=wswswsswws score=6 w_stress=4 unres_across=1 unres_within=1],
 (Parse: those|HO|urs|THAT|with.gen|TLE|work|DID.FRAME) [meter=wswswwswss score=6 w_stress=4 unres_across=2],
 (Parse: those|HO|urs|THAT|with.gen|TLE|work|DID|frame) [meter=wswswwswsw score=6 w_stress

In [24]:
mpos = parses[0].positions[0]
slot = mpos.slots[0]
unit=slot.unit
slot, unit, slot.weight_is_prom, mpos.constraint_scores

((ParseSlot) [is_stressed=True is_heavy=True is_strong=None is_weak=None],
 (Syllable: Those) [syll_ipa='ðoʊz syll_stress=P syll_weight=H is_stressed=True is_heavy=True is_strong=None is_weak=None],
 None,
 {'w_stress': 1,
  's_unstress': None,
  'unres_across': nan,
  'unres_within': nan,
  'w_peak': 0,
  's_trough': nan})

In [25]:
unres_within(mpos)

nan

In [26]:
unit.is_heavy

True